# 00. Module import & Data preprocessing

## 00 -1. Module import

In [ ]:
# Module import
!pip install soynlp
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import konlpy
from konlpy.tag import Mecab, Okt

### 00-2. data preprocesing

In [ ]:
# oepn data
data = r'/content/drive/MyDrive/AI+LION/fin_project/csv/감성대화말뭉치(최종데이터)_Training.xlsx'
chat = pd.read_excel(data)
chat.drop('Unnamed: 0',  axis = 1, inplace = True)
chat.head()

,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템문장1,사람문장2,시스템문장2,사람문장3,시스템문장3
0,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?,그냥 내가 해결하는 게 나아. 남들한테 부담 주고 싶지도 않고.,혼자 해결하기로 했군요. 혼자서 해결하기 힘들면 주위에 의논할 사람을 찾아보세요.,NaN,NaN
1,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?,최대한 지출을 억제해야겠어. 월급이 줄어들었으니 고정지출을 줄일 수밖에 없을 것 같아.,월급이 줄어든 만큼 소비를 줄일 계획이군요.,NaN,NaN
2,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...,잘 안 맞는 사람이랑 억지로 잘 지내는 것보단 조금은 거리를 두고 예의를 갖춰서 대...,스트레스받지 않기 위해선 인간관계에 있어 약간의 거리를 두는 게 좋겠군요.,NaN,NaN
3,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...,직장 사람들과 솔직하게 이야기해보고 싶어. 일하는 데에 방해된다고.,직장 사람들과 이야기를 해 보겠다고 결심하셨군요.,NaN,NaN
4,청년,여성,"진로,취업,직장",해당없음,분노,노여워하는,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...,상사인 나에게 먼저 인사하지 않아서 매일 내가 먼저 인사한다고!,항상 먼저 인사하게 되어 화가 나셨군요. 어떻게 하면 신입사원에게 화났음을 표현할 ...,NaN,NaN


In [ ]:
# data info
chat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연령      51630 non-null  object
 1   성별      51630 non-null  object
 2   상황키워드   51630 non-null  object
 3   신체질환    51630 non-null  object
 4   감정_대분류  51630 non-null  object
 5   감정_소분류  51630 non-null  object
 6   사람문장1   51630 non-null  object
 7   시스템문장1  51630 non-null  object
 8   사람문장2   51630 non-null  object
 9   시스템문장2  51630 non-null  object
 10  사람문장3   42695 non-null  object
 11  시스템문장3  42695 non-null  object
dtypes: object(12)
memory usage: 4.7+ MB


In [ ]:
# data extraction
df_chat = chat[['감정_대분류', '사람문장1', '사람문장2', '사람문장3']]

# '상처' drop
df_chat = df_chat[df_chat['감정_대분류'] != '상처']

# '사람문장1' , '사람문장2' concat & drop '사람문장3'
df_chat['합친사람문장'] = df_chat['사람문장1'] + ' ' + df_chat['사람문장2']
df_chat_merge = df_chat[['감정_대분류', '합친사람문장']]
df_chat_merge

,감정_대분류,합친사람문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...
...,...,...
51622,분노,건강관리를 너무 안 해서 건강이 좋지 않아 졌어. 주변 사람들에게 폐 끼칠까 봐 걱...
51623,당황,이제 몸이 점점 약해진다는 게 느껴져. 아내에게 미안하고 속상한 마음이 들어. 이젠...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고....
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나. 마음...


In [ ]:
df_chat_merge['감정_대분류'].value_counts()

불안    9320
분노    9160
슬픔    9125
당황    8756
기쁨    6126
Name: 감정_대분류, dtype: int64

-----------

# 01. Text preprocessing

## 01 - 1 Tokenization

In [ ]:
# Remove spaces, dots, question marks, and single quotes
df_chat_merge['문장'] = df_chat_merge['합친사람문장'].str.replace("[ .?'\"!]+", "", regex=True)
df_chat_merge.drop(columns=['합친사람문장'], inplace=True)
df_chat_merge

<ipython-input-11-e4db3a0e6009>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chat_merge['문장'] = df_chat_merge['합친사람문장'].str.replace("[ .?'\"!]+", "", regex=True)
<ipython-input-11-e4db3a0e6009>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chat_merge.drop(columns=['합친사람문장'], inplace=True)


,감정_대분류,문장
0,분노,일은왜해도해도끝이없을까화가난다그냥내가해결하는게나아남들한테부담주고싶지도않고
1,분노,이번달에또급여가깎였어물가는오르는데월급만자꾸깎이니까너무화가나최대한지출을억제해야겠어월급...
2,분노,회사에신입이들어왔는데말투가거슬려그런애를매일봐야한다고생각하니까스트레스받아잘안맞는사람이...
3,분노,직장에서막내라는이유로나에게만온갖심부름을시켜일도많은데정말분하고섭섭해직장사람들과솔직하게...
4,분노,얼마전입사한신입사원이나를무시하는것같아서너무화가나상사인나에게먼저인사하지않아서매일내가먼...
...,...,...
51622,분노,건강관리를너무안해서건강이좋지않아졌어주변사람들에게폐끼칠까봐걱정돼아내한테병시중시킬까봐미...
51623,당황,이제몸이점점약해진다는게느껴져아내에게미안하고속상한마음이들어이젠아내가날돌봐주고고생해건강...
51625,분노,나이가먹고이제돈도못벌어오니까어떻게살아가야할지막막해능력도없고아무것도할수없는내가무가치하...
51626,불안,몸이많이약해졌나봐이제전과같이일하지못할것같아너무짜증나마음같아서는다할수있는일인데이젠몸이...


In [ ]:
# Komoran
from konlpy.tag import Komoran
import pandas as pd

komoran = Komoran()

# Extract Noun, Adverb, Adjective, Verb
df_chat_merge['main_pos'] = df_chat_merge['문장'].apply(lambda x: " ".join([word_pos[0] for word_pos in komoran.pos(x) if word_pos[1] in ("NNG", "NNP", "MAG", "VA", "VV")]))
df_chat_merge.drop(columns=['문장'], inplace=True)

df_chat_merge.head()



<ipython-input-32-65bd39c0bf45>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chat_merge['main_pos'] = df_chat_merge['문장'].apply(lambda x: " ".join([word_pos[0] for word_pos in komoran.pos(x) if word_pos[1] in ("NNG", "NNP", "MAG", "VA", "VV")]))
<ipython-input-32-65bd39c0bf45>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chat_merge.drop(columns=['문장'], inplace=True)


,감정_대분류,mecab_sentence_token,main_pos
0,분노,"[(일, NNG), (은, JX), (왜, MAG), (해도, VV+EC), (해도...",일 왜 해도 해도 끝 없 가난 다그 해결 게 아남 부담 주
1,분노,"[(이번, NNG), (달, NNG), (에, JKB), (또, MAG), (급여,...",이번 달 또 급여 깎이 물가 오르 월급 자꾸 깎이 너무 화가 최 대한 지출 을 억제...
2,분노,"[(회사, NNG), (에, JKB), (신입, NNG), (이, JKS), (들어...",회사 신입 들어오 말투 슬 그러 애 매일 보 생각 스트레스 받 잘 안 맞 사람 이랑...
3,분노,"[(직장, NNG), (에서, JKB), (막내, NNG), (라는, VCP+ETM...",직장 막내 이유 로나 심부름 을 많 정말 하 해 직장 사람 과 이야기 일 방해
4,분노,"[(얼마, NNG), (전, NNG), (입사, NNG), (한, XSA+ETM),...",얼마 전 입사 한 신입사원 이나 무시 같 너무 화가 상사 인나 먼저 인사 매이 먼저 인사
